In [1]:
# kernel code for testing - https://www.kaggle.com/uchihasean/keras-cnn-band-inc-angle/code

In [2]:
data_dir = '/home/ubuntu/data/iceberg'

import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
import cv2
import keras
import os
from keras.callbacks import TensorBoard


Using TensorFlow backend.


In [3]:

np.random.seed(1234)

def get_scaled_imgs(df):
    imgs = []

    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)

        # Rescale
#         a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
#         b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
#         c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

#         imgs.append(np.dstack((a, b, c)))
        imgs.append(np.dstack((band_1, band_2, band_3)))

    return np.array(imgs)

def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

def getModel():
    # Build keras model

    image_model=Sequential()

    # CNN 1
    image_model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    image_model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    image_model.add(Dropout(0.2))

    # CNN 2
    image_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    image_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    image_model.add(Dropout(0.2))

#     # CNN 2 - added additional layers
#     image_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
#     image_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
#     image_model.add(Dropout(0.2))
    
    # CNN 3
    image_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    image_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    image_model.add(Dropout(0.2))

    # CNN 4
    image_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    image_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    image_model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    image_model.add(Flatten())
    
    print(image_model.summary())

    # Image input encoding
    image_input = Input(shape=(75,75,3))
    encoded_image = image_model(image_input)

    # Inc angle input
    inc_angle_input = Input(shape=(1,))
    
    # Combine image and inc angle
    combined= keras.layers.concatenate([encoded_image, inc_angle_input])


    dense_model = Sequential()

    # Dense 1
    dense_model.add(Dense(512, activation='relu',input_shape=(16385,)))
    dense_model.add(Dropout(0.2))

    # Dense 2
    dense_model.add(Dense(256, activation='relu'))
    dense_model.add(Dropout(0.2))

    # Output
    dense_model.add(Dense(1, activation="sigmoid"))
    
    print(dense_model.summary())

    output = dense_model(combined)

    # Final model
    combined_model= Model(inputs=[image_input,inc_angle_input],outputs= output)

    optimizer = Adam(lr=0.001, decay=0.0)
    combined_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return combined_model

df_train = pd.read_json(os.path.join(data_dir, 'train.json'))
Xtrain = get_scaled_imgs(df_train)
Ytrain = np.array(df_train['is_iceberg'])

df_train.inc_angle = df_train.inc_angle.replace('na',0)
idx_tr = np.where(df_train.inc_angle>0)

Ytrain = Ytrain[idx_tr[0]]
Xtrain = Xtrain[idx_tr[0],...]
Xinc = df_train.inc_angle[idx_tr[0]]

# remove all 3 below if you just want orginal data
# Xtrain = get_more_images(Xtrain)
# Xinc = np.concatenate((Xinc,Xinc,Xinc))
# Ytrain = np.concatenate((Ytrain,Ytrain,Ytrain))


In [4]:
model = getModel()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [25]:
batch_size = 256
        
tbCallBack = TensorBoard(log_dir='/home/ubuntu/data/tensorboardlogs/', histogram_freq=0, write_graph=True, write_images=True)
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, verbose=1, epsilon=1e-4, mode='min')



In [26]:
Xtrain.shape ,Xinc.shape, Ytrain.shape

((1471, 75, 75, 3), (1471,), (1471,))

In [27]:
model.fit([Xtrain,Xinc], Ytrain, batch_size=batch_size, epochs=10, verbose=1,
          callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tbCallBack], validation_split=0.25)

Train on 1103 samples, validate on 368 samples
Epoch 1/10
1103/1103 [==============================] - 8s 8ms/step - loss: 7.6166 - acc: 0.4878 - val_loss: 7.7525 - val_acc: 0.5190
Epoch 2/10
1103/1103 [==============================] - 8s 7ms/step - loss: 8.4171 - acc: 0.4778 - val_loss: 7.7525 - val_acc: 0.5190
Epoch 3/10
1103/1103 [==============================] - 8s 7ms/step - loss: 8.4171 - acc: 0.4778 - val_loss: 7.7525 - val_acc: 0.5190
Epoch 4/10
1103/1103 [==============================] - 8s 7ms/step - loss: 8.4171 - acc: 0.4778 - val_loss: 7.7525 - val_acc: 0.5190
Epoch 5/10
1103/1103 [==============================] - 8s 7ms/step - loss: 8.4171 - acc: 0.4778 - val_loss: 7.7525 - val_acc: 0.5190
Epoch 6/10
1103/1103 [==============================] - 8s 7ms/step - loss: 8.4171 - acc: 0.4778 - val_loss: 7.7525 - val_acc: 0.5190
Epoch 7/10
1103/1103 [==============================] - 8s 7ms/step - loss: 8.4171 - acc: 0.4778 - val_loss: 7.7525 - val_acc: 0.5190
Epoch 8/10
1103

In [ ]:

# model.load_weights(filepath = '.mdl_wts.hdf5')
score = model.evaluate([Xtrain,Xinc], Ytrain, verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

df_test = pd.read_json(os.path.join(data_dir, 'test.json'))
df_test.inc_angle = df_test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(df_test))
Xtest_inc = df_test.inc_angle
pred_test = model.predict([Xtest,Xtest_inc])



In [ ]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': pred_test.reshape((pred_test.shape[0]))})
print(submission.head(10))

submission.to_csv('cnn_train_' + str(np.around(score[1], decimals=2))  + '.csv', index=False)

In [27]:
# this model work even if I don't add the extra data, need to find out why it fits much better than 
# mine without overfitting

In [5]:
from keras.utils import plot_model
import graphviz
import pydot
plot_model(model, to_file='model.png')

In [9]:
    image_model=Sequential()

    # CNN 1
    image_model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    image_model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    image_model.add(Dropout(0.2))

    # CNN 2
    image_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    image_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    image_model.add(Dropout(0.2))

    # CNN 3
    image_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    image_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    image_model.add(Dropout(0.2))

    # CNN 4
    image_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    image_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    image_model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    image_model.add(Flatten())
    
    print(image_model.summary())

    # Image input encoding
    image_input = Input(shape=(75,75,3))
    encoded_image = image_model(image_input)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 73, 73, 64)        1792      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [54]:
Xtrain[2,:,:,0].max()

0.55959517045034968

In [15]:
image_input.get_shape()

TensorShape([Dimension(None), Dimension(75), Dimension(75), Dimension(3)])